# SDP-1 Company Name Generator

## I. Data Acqusition

_This notebook is used to collect examples of company names from WikiData using SPARQL._

***

## Table of Contents

- [Libraries](#Libraries)
- [Functions](#Functions)
- [I. Data Collection: Company Names](#I.-Data-Collection:-Company-Names)
- [II. Data Collection: Company Countries and Regions](#II.-Data-Collection:-Company-Countries-and-Regions)
- [Up next](#Up-next)

***

**WikiData Entity Examples**:
- [Apple Inc.](https://www.wikidata.org/wiki/Q312)
- [Meta Platforms](https://www.wikidata.org/wiki/Q380)

**WikiData Resources**:
- [Introduction to WikiData](https://thinking.is.ed.ac.uk/wikidata-basics/introduction-to-wikidata/)
- [SPARQL query service/queries](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries)
- [SPARQL query service/query optimization](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/query_optimization)
- [User:MartinPoulter/queries](https://www.wikidata.org/wiki/User:MartinPoulter/queries)
- [Wikidata:SPARQL tutorial](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial)

### Libraries

In [1]:
# standard library
import os
from datetime import datetime

# data wrangling
import numpy as np
import pandas as pd

# databases
from pymongo import MongoClient
from SPARQLWrapper import SPARQLWrapper, JSON

print('Loaded!')

Loaded!


In [2]:
# other settings
today = datetime.today().strftime('%y-%m-%d')
client = MongoClient(os.getenv('MONGODB_CONN'))
db = client['sdp-1']

### Functions

In [3]:
def get_results(query: str) -> pd.DataFrame:
    """
    Queries WikiData SPARQL endpoint and returns results in a table format.
    
    Parameters
    ----------
    query: str
        a SPARQL qeury.
        
    Returns
    -------
    df_lambda: pd.DataFrame
        a flat table of results.
    """
    
    # submitting a query
    endpoint = 'https://query.wikidata.org/sparql'
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    response = sparql.query().convert()
    
    # cleaning up the results
    df_lambda = pd.DataFrame(response['results']['bindings'])
    df_lambda = df_lambda.apply(lambda s: s.str.get('value'))
    
    return df_lambda

In [4]:
# testing: obtaining ids and labels of instances that belong to a class "specialised agency of the United Nations"
query = """
SELECT ?item ?itemLabel
WHERE
{
    ?item wdt:P31 wd:Q15925165. 
    OPTIONAL {?item schema:description ?desc. FILTER (lang(?desc) = "en")}
    SERVICE wikibase:label {bd:serviceParam wikibase:language "en"}
}
ORDER BY ?itemLabel
"""

get_results(query)

,item,itemLabel
0,http://www.wikidata.org/entity/Q82151,Food and Agriculture Organization
1,http://www.wikidata.org/entity/Q41984,International Atomic Energy Agency
2,http://www.wikidata.org/entity/Q191384,International Bank for Reconstruction and Deve...
3,http://www.wikidata.org/entity/Q125761,International Civil Aviation Organization
4,http://www.wikidata.org/entity/Q827525,International Development Association
5,http://www.wikidata.org/entity/Q656801,International Finance Corporation
6,http://www.wikidata.org/entity/Q689859,International Fund for Agricultural Development
7,http://www.wikidata.org/entity/Q54129,International Labour Organization
8,http://www.wikidata.org/entity/Q201054,International Maritime Organization
9,http://www.wikidata.org/entity/Q7804,International Monetary Fund


### I. Data Collection: Company Names

[To the table of contents](#Table-of-Contents)

<div class="alert alert-block alert-info">
    <ul>
        <li>In this section, I retrieve names of businesses, including alternative names and aliases from WikiData.</li>
        <li>There are two relevant classes of entities: <code>Q783794</code> (<i>"company"</i>) and <code>Q4830453</code> (<i>"business"</i>).</li>
        <li>For brevity, I focus on the latter class which is also larger and arguably more generic.</li>
        <li>The queries return not only instances of the "business" class but also instances of any of its subclasses.</li>
    </ul>
</div> 

In [5]:
# counting the number of entities in the "company" class
query = """
SELECT  (COUNT(DISTINCT ?item) AS ?entities)
WHERE
{
  ?item wdt:P31/wdt:P279* wd:Q783794.
}
"""
get_results(query)

,entities
0,307901


In [6]:
# counting the number of entities in the "business" class
query = """
SELECT  (COUNT(DISTINCT ?item) AS ?entities)
WHERE
{
  ?item wdt:P31/wdt:P279* wd:Q4830453.
}
"""
get_results(query)

,entities
0,430177


In [7]:
# matching instances of the "business" class and its subclasses that have (English) labels
query = """
SELECT ?item ?itemLabel
WHERE
{
    ?item wdt:P31/wdt:P279* wd:Q4830453. 
    ?item rdfs:label ?itemLabel. FILTER(LANG(?itemLabel) = "en")
}
"""

df_companies = get_results(query)
print('Shape:', df_companies.shape)
display(df_companies.head())

Shape: (410199, 2)


,item,itemLabel
0,http://www.wikidata.org/entity/Q11148,The Guardian
1,http://www.wikidata.org/entity/Q503176,Christie's
2,http://www.wikidata.org/entity/Q654762,Sotheby's
3,http://www.wikidata.org/entity/Q668545,Dorotheum
4,http://www.wikidata.org/entity/Q892447,Bonhams


In [8]:
# instances that belong to several (sub)classes are duplicated
print('Shape before:', df_companies.shape)
df_companies = df_companies.groupby('item', as_index = False).agg({'itemLabel': 'unique'})
assert df_companies['itemLabel'].str.len().eq(1).all()
df_companies['itemLabel'] = df_companies['itemLabel'].str.get(0)

print('Shape after :', df_companies.shape)
display(df_companies.head())

Shape before: (410199, 2)
Shape after : (343675, 2)


,item,itemLabel
0,http://www.wikidata.org/entity/P750,distributed by
1,http://www.wikidata.org/entity/Q100000067,The Loovre
2,http://www.wikidata.org/entity/Q1000042,Ships built at Meidericher Schiffswerft
3,http://www.wikidata.org/entity/Q1000076,Custom Coasters International
4,http://www.wikidata.org/entity/Q1000212,"Montreal Light, Heat & Power"


In [9]:
# obtaining alternative labels in English only
query = """
SELECT ?item ?altLabel
WHERE
{
    ?item wdt:P31/wdt:P279* wd:Q4830453. 
    ?item skos:altLabel ?altLabel. FILTER (lang(?altLabel) = "en")
}
"""

df_lambda = get_results(query)
print('Shape:', df_lambda.shape)
display(df_lambda.head())

Shape: (184214, 2)


,item,altLabel
0,http://www.wikidata.org/entity/Q493319,Min Aigi
1,http://www.wikidata.org/entity/Q494170,Washington Times
2,http://www.wikidata.org/entity/Q494170,washingtontimes.com
3,http://www.wikidata.org/entity/Q497170,경향신문
4,http://www.wikidata.org/entity/Q498921,Straits Times


In [10]:
# aggregating to the entity level, collecting alternative labels into an array
print('Shape before:', df_lambda.shape)
df_lambda = df_lambda.groupby('item', as_index = False).agg({'altLabel': 'unique'})
print('Shape after :', df_lambda.shape)
display(df_lambda.head())

Shape before: (184214, 2)
Shape after : (90972, 2)


,item,altLabel
0,http://www.wikidata.org/entity/P750,[distributor]
1,http://www.wikidata.org/entity/Q100000067,"[Bankhill Ladies Toilet, Northern Edge Coffee ..."
2,http://www.wikidata.org/entity/Q1000076,"[CCI, Custom Coasters International, Inc.]"
3,http://www.wikidata.org/entity/Q1000212,"[MLH&P, Montreal Light, Heat and Power Company]"
4,http://www.wikidata.org/entity/Q100042496,"[Theatre de l'OEil, Theatre de l'Œil]"


In [11]:
# merging with the main dataframe
print('Shape before:', df_companies.shape)
df_companies = df_companies.merge(df_lambda, on = 'item', how = 'left')
print('Shape after :', df_companies.shape)
display(df_companies.head())

Shape before: (343675, 2)
Shape after : (343675, 3)


,item,itemLabel,altLabel
0,http://www.wikidata.org/entity/P750,distributed by,[distributor]
1,http://www.wikidata.org/entity/Q100000067,The Loovre,"[Bankhill Ladies Toilet, Northern Edge Coffee ..."
2,http://www.wikidata.org/entity/Q1000042,Ships built at Meidericher Schiffswerft,NaN
3,http://www.wikidata.org/entity/Q1000076,Custom Coasters International,"[CCI, Custom Coasters International, Inc.]"
4,http://www.wikidata.org/entity/Q1000212,"Montreal Light, Heat & Power","[MLH&P, Montreal Light, Heat and Power Company]"


### II. Data Collection: Company Countries and Regions

[To the table of contents](#Table-of-Contents)

<div class="alert alert-block alert-info">
    <ul>
        <li>In this section, I collect information on countries the business entities are associated with.</li>
        <li>These countries are then linked to 5 regions using <a href="https://unstats.un.org/unsd/methodology/m49/overview/" target=_blank>M49</a>, the classification provided by the UN Statistics Division.</li>
        <li>The new dimension can be used to generate company names that are conditioned on the region of origin, i.e. one can produce names that sound a bit more Asian or European.</li>
    </ul>
</div> 

In [12]:
# obtaining entity countries
query = """
SELECT ?item ?country
WHERE
{
    ?item wdt:P31/wdt:P279* wd:Q4830453; 
    wdt:P17 ?country.
}
"""

df_lambda = get_results(query)
print('Shape:', df_lambda.shape)
display(df_lambda.head())

Shape: (362482, 2)


,item,country
0,http://www.wikidata.org/entity/Q492825,http://www.wikidata.org/entity/Q884
1,http://www.wikidata.org/entity/Q494170,http://www.wikidata.org/entity/Q30
2,http://www.wikidata.org/entity/Q494902,http://www.wikidata.org/entity/Q36
3,http://www.wikidata.org/entity/Q496011,http://www.wikidata.org/entity/Q142
4,http://www.wikidata.org/entity/Q497170,http://www.wikidata.org/entity/Q884


In [13]:
# obtaining country labels and ISO codes
query = """
SELECT ?country ?countryLabel ?iso3Label
WHERE
{
    ?country wdt:P31 wd:Q6256;
    wdt:P298 ?iso3
    SERVICE wikibase:label {bd:serviceParam wikibase:language "en"}
}
"""

df_countries = get_results(query)
print('Shape:', df_countries.shape)
display(df_countries.head())

Shape: (177, 3)


,country,countryLabel,iso3Label
0,http://www.wikidata.org/entity/Q810,Jordan,JOR
1,http://www.wikidata.org/entity/Q811,Nicaragua,NIC
2,http://www.wikidata.org/entity/Q813,Kyrgyzstan,KGZ
3,http://www.wikidata.org/entity/Q819,Laos,LAO
4,http://www.wikidata.org/entity/Q822,Lebanon,LBN


In [14]:
# adding country labels and iso codes, keeping only the entries where country name and iso are available
print('Shape before:', df_lambda.shape)
df_lambda = df_lambda.merge(df_countries, on = 'country', how = 'inner'); del df_countries
print('Shape after :', df_lambda.shape)
display(df_lambda.head())

Shape before: (362482, 2)
Shape after : (326211, 4)


,item,country,countryLabel,iso3Label
0,http://www.wikidata.org/entity/Q492825,http://www.wikidata.org/entity/Q884,South Korea,KOR
1,http://www.wikidata.org/entity/Q497170,http://www.wikidata.org/entity/Q884,South Korea,KOR
2,http://www.wikidata.org/entity/Q483924,http://www.wikidata.org/entity/Q884,South Korea,KOR
3,http://www.wikidata.org/entity/Q486314,http://www.wikidata.org/entity/Q884,South Korea,KOR
4,http://www.wikidata.org/entity/Q489308,http://www.wikidata.org/entity/Q884,South Korea,KOR


In [15]:
# skips "Bonaire, Sint Eustatius and Saba" due to a comma in the name
df_countries = pd.read_csv(
    '../datain/UNSD — Methodology.csv', # https://unstats.un.org/unsd/methodology/m49/overview/
    sep = ',',
    on_bad_lines = 'skip'
)
print('Shape:', df_countries.shape)
display(df_countries.head())

Shape: (246, 15)


,Global Code,Global Name,Region Code,Region Name,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Country or Area,M49 Code,ISO-alpha2 Code,ISO-alpha3 Code,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS)
0,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Algeria,12,DZ,DZA,NaN,NaN,NaN
1,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Egypt,818,EG,EGY,NaN,NaN,NaN
2,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Libya,434,LY,LBY,NaN,NaN,NaN
3,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Morocco,504,MA,MAR,NaN,NaN,NaN
4,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Sudan,729,SD,SDN,x,NaN,NaN


In [16]:
# Channel Islands  have no ISO, so it will be removed
df_countries.loc[df_countries['ISO-alpha3 Code'].isna()]

,Global Code,Global Name,Region Code,Region Name,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Country or Area,M49 Code,ISO-alpha2 Code,ISO-alpha3 Code,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS)
178,1,World,150.0,Europe,154.0,Northern Europe,830.0,Channel Islands,Sark,680,NaN,NaN,NaN,NaN,NaN


In [17]:
# cleaning up the dataset
to_rename = {
    'Region Name': 'region',
    'ISO-alpha3 Code': 'iso3Label'
}
print('Shape before:', df_countries.shape)

df_countries.dropna(subset = ['ISO-alpha3 Code'], inplace = True)
df_countries = df_countries.rename(to_rename, axis = 1).reindex(to_rename.values(), axis = 1)

print('Shape after :', df_countries.shape)
display(df_countries.head())

Shape before: (246, 15)
Shape after : (245, 2)


,region,iso3Label
0,Africa,DZA
1,Africa,EGY
2,Africa,LBY
3,Africa,MAR
4,Africa,SDN


In [18]:
df_countries['region'].value_counts()

Africa      60
Americas    56
Europe      51
Asia        48
Oceania     29
Name: region, dtype: int64

In [19]:
# merging regions and countries data
print('Shape before:', df_lambda.shape)
df_lambda = df_lambda.merge(df_countries, on = 'iso3Label', how = 'left'); del df_countries
print('Shape after :', df_lambda.shape)
display(df_lambda.head())

Shape before: (326211, 4)
Shape after : (326211, 5)


,item,country,countryLabel,iso3Label,region
0,http://www.wikidata.org/entity/Q492825,http://www.wikidata.org/entity/Q884,South Korea,KOR,Asia
1,http://www.wikidata.org/entity/Q497170,http://www.wikidata.org/entity/Q884,South Korea,KOR,Asia
2,http://www.wikidata.org/entity/Q483924,http://www.wikidata.org/entity/Q884,South Korea,KOR,Asia
3,http://www.wikidata.org/entity/Q486314,http://www.wikidata.org/entity/Q884,South Korea,KOR,Asia
4,http://www.wikidata.org/entity/Q489308,http://www.wikidata.org/entity/Q884,South Korea,KOR,Asia


In [20]:
df_lambda['region'].value_counts(dropna = False)

Europe      199898
Americas     74379
Asia         42087
Oceania       5059
Africa        3911
NaN            877
Name: region, dtype: int64

In [21]:
# Taiwan has got no region since it is not in the M49 list
df_lambda.loc[df_lambda['region'].isna(), ['countryLabel', 'iso3Label', 'region']].drop_duplicates()

,countryLabel,iso3Label,region
261246,Taiwan,TWN,NaN


In [22]:
# adding the region manually to all companies from Taiwan
df_lambda['region'].fillna('Asia', inplace = True)

In [23]:
# collecting country/region locations into an array since some entities have multiple entries
print('Shape before:', df_lambda.shape)
df_lambda = df_lambda.groupby('item', as_index = False).agg({'iso3Label': 'unique', 'region': 'unique'})
print('Shape after :', df_lambda.shape)
display(df_lambda.head())

Shape before: (326211, 5)
Shape after : (263519, 3)


,item,iso3Label,region
0,http://www.wikidata.org/entity/Q100000067,[GBR],[Europe]
1,http://www.wikidata.org/entity/Q1000042,[DEU],[Europe]
2,http://www.wikidata.org/entity/Q1000076,[USA],[Americas]
3,http://www.wikidata.org/entity/Q1000317,[DEU],[Europe]
4,http://www.wikidata.org/entity/Q100042496,[CAN],[Americas]


In [24]:
# most companies are associated with only one country but some have more than one
df_lambda['iso3Label'].str.len().value_counts().sort_index()

1     262798
2        562
3         75
4         27
5         14
6         14
7          8
8          6
9          1
10         4
11         2
12         5
13         1
14         1
19         1
Name: iso3Label, dtype: int64

In [25]:
# likewise, some companies are linked to multiple regions
df_lambda['region'].str.len().value_counts().sort_index()

1    263192
2       294
3        22
4         9
5         2
Name: region, dtype: int64

In [26]:
# finally, merging country and regions with companies
print('Shape before:', df_companies.shape)
df_companies = df_companies.merge(df_lambda, on = 'item', how = 'left')
print('Shape after :', df_companies.shape)
display(df_companies.head())

Shape before: (343675, 3)
Shape after : (343675, 5)


,item,itemLabel,altLabel,iso3Label,region
0,http://www.wikidata.org/entity/P750,distributed by,[distributor],NaN,NaN
1,http://www.wikidata.org/entity/Q100000067,The Loovre,"[Bankhill Ladies Toilet, Northern Edge Coffee ...",[GBR],[Europe]
2,http://www.wikidata.org/entity/Q1000042,Ships built at Meidericher Schiffswerft,NaN,[DEU],[Europe]
3,http://www.wikidata.org/entity/Q1000076,Custom Coasters International,"[CCI, Custom Coasters International, Inc.]",[USA],[Americas]
4,http://www.wikidata.org/entity/Q1000212,"Montreal Light, Heat & Power","[MLH&P, Montreal Light, Heat and Power Company]",NaN,NaN


In [27]:
# cleaning up
print('Shape before:', df_companies.shape)

df_companies['item'] = df_companies['item'].str.split('/').str.get(-1)
df_companies.drop(0, inplace = True) # removes P750
assert df_companies['item'].str.match('Q\d+').all(), 'Some entries are not WikiData entities.'
df_companies = df_companies.where(pd.notna, None)

# converting numpy arrays obtained during aggregation to lists
for x in ['altLabel', 'iso3Label', 'region']:
    df_companies[x] = df_companies[x].apply(lambda x: x if x is None else x.tolist())
    
print('Shape after :', df_companies.shape)
display(df_companies.head())

Shape before: (343675, 5)
Shape after : (343674, 5)


,item,itemLabel,altLabel,iso3Label,region
1,Q100000067,The Loovre,"[Bankhill Ladies Toilet, Northern Edge Coffee ...",[GBR],[Europe]
2,Q1000042,Ships built at Meidericher Schiffswerft,None,[DEU],[Europe]
3,Q1000076,Custom Coasters International,"[CCI, Custom Coasters International, Inc.]",[USA],[Americas]
4,Q1000212,"Montreal Light, Heat & Power","[MLH&P, Montreal Light, Heat and Power Company]",None,None
5,Q100032157,Svatek a spol.,None,None,None


In [28]:
# inserting the records into MongoDB
collection = db[f'companies-v{today}']
collection.insert_many(df_companies.to_dict(orient = 'records'))

### Up next

[To the table of contents](#Table-of-Contents)

<div class="alert alert-block alert-info">
    <ul>
        <li>In the next notebook, I prepare the data for model training by reshaping and cleaning it up.</li>
        <li>There, I also explore the dataset in greater detail and produce several visualisations.</li>
    </ul>
</div> 